<a href="https://colab.research.google.com/github/Pataweepr/applyML_vistec_2019/blob/master/hw3_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Titanic

พวกเราจะพยายามให้ logictic reg เพื่อทำนาย แบบจำลอง

In [0]:
# import library 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LinearRegression¶
from google.colab import files
# from sklearn import datasets

# load data from kaggle 
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url) #training set
test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url) #test set


In [0]:
################################################################################
# TODO#1:                                                                      #
# clean your data
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

In [0]:
################################################################################
# TODO#2: using scikit learn creat Logistic regression model 
# and fit it with data_train
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################


[logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [0]:
# ดูค่า precision recall ที่หาได้
################################################################################
# TODO#3: using scikit learn creat Logistic regression model 
# and fit it with data_train
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

## submit your result

In [0]:
################################################################################
# TODO#4:                                                                      #
# using the model with your test set
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

logreg.predict_proba(X[:2, :]) 

In [0]:
################################################################################
# TODO#5:  Submit a screenshot of your submission (with the scores)
# Warning! check your format before submit to kaggle
# example data type
################################################################################


## RE-do with linear regression model 
[link text](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [0]:
################################################################################
# TODO#6: solving linear regression model                                                                   #
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

## compare result

## select feature
 What happens if you reduce the amount of features to just Sex and Age? (Use logistic regression)


In [0]:
################################################################################
# TODO#7: RE-do with linear regression model                                                                   #
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

## created complex feature

Try adding some higher order features to your training (x2 1,x1x2,...). 

In [0]:
################################################################################
# TODO#8: RE-do with linear regression model                                                                   #
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################